In [1]:
import pandas as pd

from df.keiba_data_frame import KeibaDataFrame
from df.result_frame import ResultFrame
from env.reward import Reward

# Train Data 読み込み
# ※ 既に AI 側で filter/data 整備は実施済みのため再 type 明確化と、特定列を Drop のみ行っている。
kdf = KeibaDataFrame.from_data("./../data/train/2004_ranking_without_g.csv")
kdf.setting_types()

# Reward 準備
reward = Reward.initialize("./../data/train/2004_result.csv")

In [2]:
import numpy as np
from tensorflow import keras

new_model = keras.models.load_model("./../model/ranking_model_without_g")

preds = []

i = 0
for v in kdf.groups():
    obs = v.reindex(range(20), fill_value=-1).values.reshape(1, 1, 20, 51).tolist()
    q_values = new_model.predict(obs)
    action = np.argmax(q_values[0])
    pred_df = pd.DataFrame(
        {
            "track_id": v["track_id"],
            "date_num": v["date_num"],
            "round": v["round"],
            "number": v["number"],
            "pred": v["pred"],
            "action": action,
        }
    )
    preds.append(pred_df)
    i += 1
    if i > 100:
        break

preds_df = pd.concat(preds)

1/1 [==============================] - 0s 31ms/step


In [9]:
mikounyu_count = 0
kounyu_count = 0
atari_count = 0
hazure_count = 0
reward_total = 0

action_0_count = 0
action_1_count = 0
action_2_count = 0
action_3_count = 0
action_4_count = 0

df = preds_df.sort_values(
    ["date_num", "track_id", "round"], ascending=[True, True, True]
)

for _, v in df.groupby(["date_num", "track_id", "round"], sort=False):
    action = int(v.iloc[0]["action"])

    if action == 0:
        action_0_count += 1
    elif action == 1:
        action_1_count += 1
    elif action == 2:
        action_2_count += 1
    elif action == 3:
        action_3_count += 1
    elif action == 4:
        action_4_count += 1

    if action == 5:
        reward_total += 0
        mikounyu_count += 1
    else:
        kounyu_count += 1
        result = reward.calc(v, action)
        if result > 0:
            atari_count += 1
        else:
            hazure_count += 1

        reward_total += result

print(f"RANK_ONE_HORSE 回数: {action_0_count}")
print(f"RANK_TWO_HORSE 回数: {action_1_count}")
print(f"RANK_THREE_HORSE 回数: {action_2_count}")
print(f"RANK_FOUR_HORSE 回数: {action_3_count}")
print(f"RANK_FIVE_HORSE 回数: {action_4_count}")
print(f"未購入回数: {mikounyu_count}")
print(f"当たり馬券: {atari_count}")
print(f"はずれ馬券: {hazure_count}")
print(f"的中率: {(atari_count / (atari_count + hazure_count))}")
print(f"儲け: {reward_total}")

    track_id    date_num  round  number       pred  action
0          4  1073228400      1       1  -6.830237       5
1          4  1073228400      1       2  -7.470257       5
2          4  1073228400      1       3  -4.276666       5
3          4  1073228400      1       4 -10.138005       5
4          4  1073228400      1       5  -8.974765       5
5          4  1073228400      1       6  -5.953210       5
6          4  1073228400      1       7  -8.284420       5
7          4  1073228400      1       8  -7.412075       5
8          4  1073228400      1       9  -7.904926       5
9          4  1073228400      1      10  -8.737642       5
10         4  1073228400      1      11  -6.263238       5
11         4  1073228400      1      12  -8.757496       5
12         4  1073228400      1      13  -7.810339       5
13         4  1073228400      1      14  -7.556525       5
14         4  1073228400      1      15  -9.228726       5
15         4  1073228400      1      16  -6.269521      